In [1]:
import warnings
warnings.filterwarnings("ignore")
# !pip install ftfy
import ftfy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
from math import exp
from numpy import sign
import sys
print(sys.executable)
from PIL import Image # getting images in notebook
# !pip install gensim
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# !pip install tensorflow

# !pip install tensorflow_hub

# !pip install bert-for-tf2
# !pip install sentencepiece

C:\Users\SHWETANK VERMA\Anaconda3\latest\python.exe


[nltk_data] Downloading package stopwords to C:\Users\SHWETANK
[nltk_data]     VERMA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\SHWETANK
[nltk_data]     VERMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# from tensorflow import keras 
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
import re
from tqdm import tqdm
import tensorflow as tf
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [3]:
train=pd.read_csv(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\Datasets\train.csv')

In [4]:
train.sample(3)

,PID,Text_data,Label
2794,train_pid_2795,I think I can't be happy without a relationshi...,moderate
4548,train_pid_4549,"Starting the new year off ""right"" : First day ...",moderate
7536,train_pid_7537,Time is just one big blur when you have depres...,not depression


In [5]:
test=pd.read_csv(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\Datasets\dev.csv')

In [6]:
print("Training set has {} rows and {} columns.".format(train.shape[0], train.shape[1]))
print("Test set has {} rows and {} columns.".format(test.shape[0], test.shape[1]))

print()
print(train.columns)
print(test.columns)

Training set has 8891 rows and 3 columns.
Test set has 4496 rows and 3 columns.

Index(['PID', 'Text_data', 'Label'], dtype='object')
Index(['PID', 'Text data', 'Label'], dtype='object')


In [7]:
Sentiment = {'moderate': 1,'not depression': 0,'severe':2}
train.Label = [Sentiment[item] for item in train.Label]
test.Label= [Sentiment[item] for item in test.Label]

In [8]:
train.sample(3)

,PID,Text_data,Label
1187,train_pid_1188,Ayy sausage body is back in the game : Does an...,1
187,train_pid_188,"Jeez, looking back at the decade is horribly d...",1
6302,train_pid_6303,Trying to avoid social media. If anyone wants ...,0


In [9]:
test.sample(3)


,PID,Text data,Label
3890,dev_pid_3891,"Support group : Hi everyone, I’ve just joined ...",0
608,dev_pid_609,"Lost my sparks for drawing : You know, i've be...",1
1887,dev_pid_1888,Just need to post so I don’t feel so alone.. :...,1


In [10]:
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# NLTK Tweet Tokenizer for now
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

corpus = []

# clean up text
def clean_text(text):
    """
    Copied from other notebooks
    """
    # expand acronyms
    
    # special characters
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    # emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    
    """
    Our Stuff
    """
    # remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    # remove punctuation and special chars (keep '!')
    for p in string.punctuation.replace('!', ''):
        text = text.replace(p, '')
        
    # remove urls
    text = re.sub(r'http\S+', '', text)
    
    # tokenize
    text = tknzr.tokenize(text)
    
    # remove stopwords
    text = [w.lower() for w in text if not w in stop_words]
    corpus.append(text)
    
    # join back
    text = ' '.join(text)
    
    return text

In [11]:
%%time
train['Text_data'] = train['Text_data'].apply(lambda s: clean_text(s))
test['Text data'] = test['Text data'].apply(lambda s: clean_text(s))

# see some cleaned data
train.sample(10)

Wall time: 30.5 s


,PID,Text_data,Label
8659,train_pid_8660,help im new reddit i dont really know works i ...,2
4995,train_pid_4996,conflicted i know i go social part wants i als...,1
1467,train_pid_1468,how everyone ’ new year oh mine something else...,1
2200,train_pid_2201,for last months life falling apart im my girlf...,1
3492,train_pid_3493,feeling alone nye party i went first real nye ...,1
1144,train_pid_1145,i ’ depressed high i need someone life love ev...,1
3020,train_pid_3021,happy new year it new year everyone takes deci...,1
6642,train_pid_6643,how i nonreligious help girlfriend religious g...,0
8827,train_pid_8828,don ’ know what do anymore meds ’ worked thera...,2
8831,train_pid_8832,what wrong do ever lie wondering actually wron...,2


In [12]:
texts = train['Text_data'].to_numpy()
word_freq = {}

for text in texts:
    for word in text.split():
        word_freq[word] = word_freq.get(word, 0) + 1

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH = 40

tokenizer = Tokenizer()

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = len(word_index) + 1
print('Found %s unique tokens.' % (num_words - 1))

# pad 
data = pad_sequences(
    sequences, 
    maxlen=MAX_SEQUENCE_LENGTH,
    padding='post', 
    truncating='post'
)

labels = train['Label'].to_numpy()
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 13266 unique tokens.
Shape of data tensor: (8891, 40)
Shape of label tensor: (8891,)


In [14]:
x_train = data
y_train = labels

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.metrics import roc_auc_score

vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(train['Text_data'])

# print vocabulary
print(vectorizer.get_feature_names()[2500:2600])

['courageous', 'course', 'courses', 'coursework', 'court', 'courtesy', 'cousin', 'cousindads', 'cousins', 'cousy', 'cover', 'coverage', 'covered', 'covering', 'covers', 'covid', 'cow', 'coward', 'cowardice', 'cowards', 'cower', 'cowered', 'coworker', 'coworkers', 'coz', 'cozy', 'cps', 'cptsd', 'crab', 'crack', 'cracked', 'cracking', 'cracks', 'cradle', 'cradled', 'craft', 'crafts', 'craig', 'craigslist', 'cramps', 'crap', 'crappier', 'crappy', 'crash', 'crashes', 'crashing', 'crave', 'craved', 'craving', 'cravings', 'crawl', 'crawled', 'crawling', 'crazed', 'craziest', 'crazy', 'crazyo', 'cream', 'create', 'created', 'creates', 'creating', 'creation', 'creations', 'creative', 'creativity', 'creator', 'creators', 'creature', 'creatures', 'credit', 'creep', 'creeped', 'creeping', 'creeps', 'creepy', 'crept', 'crevice', 'crib', 'crickets', 'cried', 'cries', 'crieswhy', 'crime', 'crimes', 'criminal', 'cringe', 'cringey', 'cringing', 'cringy', 'cripple', 'crippled', 'cripples', 'crippling',

In [16]:
x = train["Text_data"]
y = train["Label"]

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

weather_encoded = le.fit_transform(x)


label=le.fit_transform(y)

In [19]:
features = list(weather_encoded)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [23]:
knn = KNeighborsClassifier(n_neighbors=7)

In [24]:
clf = knn.fit(xv_train, y_train)

In [25]:
def prediction(X_test, model_object):
    y_pred = model_object.predict(xv_test)
    print("Predicted values:")
    print(y_pred)
    return y_pred

In [26]:
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: ",
        confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

In [27]:
y_pred = prediction(xv_test, clf)

Predicted values:
[0 0 1 ... 1 1 0]


In [28]:
cal_accuracy(y_test, y_pred)
print(classification_report(y_test,y_pred))

Confusion Matrix:  [[ 298  176   19]
 [  46 1419   18]
 [  11   96  140]]
Accuracy :  83.53576248313091
              precision    recall  f1-score   support

           0       0.84      0.60      0.70       493
           1       0.84      0.96      0.89      1483
           2       0.79      0.57      0.66       247

    accuracy                           0.84      2223
   macro avg       0.82      0.71      0.75      2223
weighted avg       0.83      0.84      0.83      2223



In [29]:
# using count vectorizer
vectorizer = CountVectorizer()

xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)

In [30]:
clf = knn.fit(xv_train, y_train)
y_pred = prediction(xv_test, clf)
cal_accuracy(y_test, y_pred)
print(classification_report(y_test,y_pred))

Predicted values:
[0 0 1 ... 1 1 0]
Confusion Matrix:  [[ 352  131   10]
 [ 103 1357   23]
 [  33   75  139]]
Accuracy :  83.13090418353576
              precision    recall  f1-score   support

           0       0.72      0.71      0.72       493
           1       0.87      0.92      0.89      1483
           2       0.81      0.56      0.66       247

    accuracy                           0.83      2223
   macro avg       0.80      0.73      0.76      2223
weighted avg       0.83      0.83      0.83      2223



In [31]:
#Using Pretained Embeddings 

In [32]:
#using word2vec

In [46]:
#Splitting the data into dependent and independent variables i.e, features and the target columns
X=train['Text_data']
Y=train['Label']
#Splitting the data such that 33% will be used for testing and the remaining 67% will be used for training. 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,stratify=Y,test_size=0.33)

In [34]:
from gensim.models import Word2Vec

In [35]:
words_in_sentences=[]
for i in tqdm(x_train):
    words_in_sentences.append(i.split())

100%|███████████████████████████████████████████████████████████████████████████| 5956/5956 [00:00<00:00, 67046.19it/s]


In [36]:
model = Word2Vec(sentences=words_in_sentences, vector_size=200,workers=-1)

In [37]:
vocab=list(model.wv.key_to_index.keys())
print(len(vocab))

6026


In [38]:
def avg_w2vec(sentences):
    transformed=[]
    for sentence in tqdm(sentences):
        count=0
        vector=np.zeros(200)
        for word in sentence.split():
            if word in vocab:
                vector+=model.wv.get_vector(word)
                count+=1
        if count!=0:
            vector/=count
            transformed.append(vector)
        else:
            print(sentence)
    return np.array(transformed)

In [39]:
x_train_transformed=avg_w2vec(x_train)
x_test_transformed=avg_w2vec(x_test)

 71%|██████████████████████████████████████████████████████▍                      | 2075/2935 [00:09<00:04, 201.88it/s]

fuckfuckfuckfuckfuck


100%|█████████████████████████████████████████████████████████████████████████████| 2935/2935 [00:11<00:00, 266.20it/s]


In [40]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
grid_params = { 'n_neighbors' : [10,20,30,40,50,60],
               'metric' : ['manhattan']}
knn=KNeighborsClassifier()
clf = RandomizedSearchCV(knn, grid_params, random_state=0,n_jobs=-1,verbose=1)

In [41]:
x_train_transformed.shape[0] 

5956

In [47]:
y_train.shape[0]

5956

In [48]:
y_train=y_train

In [49]:
y_train.shape[0]

5956

In [50]:
clf.fit(x_train_transformed,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


RandomizedSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'metric': ['manhattan'],
                                        'n_neighbors': [10, 20, 30, 40, 50,
                                                        60]},
                   random_state=0, verbose=1)

In [51]:
clf.best_score_

0.6752854180402232

In [52]:
print(classification_report(y_train,clf.predict(x_train_transformed)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1330
           1       0.68      1.00      0.81      4022
           2       0.00      0.00      0.00       604

    accuracy                           0.68      5956
   macro avg       0.23      0.33      0.27      5956
weighted avg       0.46      0.68      0.54      5956



In [57]:
y_test.shape[0]
y_test=y_test[1:]


In [56]:
x_test_transformed.shape[0]

2934

In [58]:
print(classification_report(y_test,clf.predict(x_test_transformed)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       655
           1       0.68      1.00      0.81      1981
           2       0.00      0.00      0.00       298

    accuracy                           0.68      2934
   macro avg       0.23      0.33      0.27      2934
weighted avg       0.46      0.68      0.54      2934



In [59]:
!pip install transformers

     ---------------------------------------- 5.5/5.5 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 182.4/182.4 kB 1.6 MB/s eta 0:00:00
